In [1]:
import pandas as pd
import numpy as np

file_path = '../data/warfarin.csv'

try:
    df = pd.read_csv(file_path)

    # Check the dimensions of the dataset
    print("Dataset shape:")
    print(df.shape)

    # Display the first 5 rows to confirm it loaded correctly
    print("\nFirst 5 rows:")
    display(df.head())
    
except Exception as e:
    print(f"An error occurred: {e}")
    print("\nPlease double-check that the file 'warfarin.csv' is saved correctly inside the 'data' folder.")

Dataset shape:
(843, 69)

First 5 rows:


,PharmGKB Subject ID,Gender,Race,Ethnicity,Age,Height (cm),Weight (kg),Indication for Warfarin Treatment,Comorbidities,Medications,...,VKORC1-1639.1,VKORC1-1639.2,VKORC1-1639.3,VKORC1-1639.4,VKORC1-1639.5,VKORC1-1639.6,VKORC1-1639.7,VKORC1-1639.8,VKORC1-1639.9,VKORC1-1639.10
0,PA128919639,male,White,Not Hispanic or Latino,60 - 69,170.18,103.4,Pulmonary Embolism,Diabetes;Hypothyroidism,levothyroxine;metformin,...,0,1,0,0,0,0,0,0,0.0,0.0
1,PA128919639,male,White,Not Hispanic or Latino,60 - 69,170.18,103.4,Pulmonary Embolism,Diabetes;Hypothyroidism,levothyroxine;metformin,...,0,1,0,0,0,0,0,0,0.0,0.0
2,PA128919639,male,White,Not Hispanic or Latino,60 - 69,170.18,103.4,Pulmonary Embolism,Diabetes;Hypothyroidism,levothyroxine;metformin,...,0,1,0,0,0,0,0,0,0.0,0.0
3,PA128919639,male,White,Not Hispanic or Latino,60 - 69,170.18,103.4,Pulmonary Embolism,Diabetes;Hypothyroidism,levothyroxine;metformin,...,0,1,0,0,0,0,0,0,0.0,0.0
4,PA128919639,male,White,Not Hispanic or Latino,60 - 69,170.18,103.4,Pulmonary Embolism,Diabetes;Hypothyroidism,levothyroxine;metformin,...,0,1,0,0,0,0,0,0,0.0,0.0


In [2]:
# Get a high-level summary of the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 843 entries, 0 to 842
Data columns (total 69 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   PharmGKB Subject ID                                              843 non-null    object 
 1   Gender                                                           843 non-null    object 
 2   Race                                                             843 non-null    object 
 3   Ethnicity                                                        843 non-null    object 
 4   Age                                                              843 non-null    object 
 5   Height (cm)                                                      843 non-null    float64
 6   Weight (kg)                                                      843 non-null    float64
 7   Indication for Warfarin Treatment           

In [3]:
# Calculate the number of missing values for each column
missing_values = df.isnull().sum()

# Filter and display only the columns that have one or more missing values
print("Columns with missing values:")
print(missing_values[missing_values > 0])

Columns with missing values:
Comorbidities     525
VKORC1-1639.9       1
VKORC1-1639.10      1
dtype: int64


In [4]:
# Print the shape of the DataFrame before cleaning
print(f"Shape before cleaning: {df.shape}")

# Step 1: Drop the 'Comorbidities' column
df.drop(columns=['Comorbidities'], inplace=True)
print("Dropped 'Comorbidities' column.")

# Step 2 & 3: Drop rows where our target variable or the minor columns are missing
columns_to_check = ['Therapeutic Dose of Warfarin', 'VKORC1-1639.9', 'VKORC1-1639.10']
df.dropna(subset=columns_to_check, inplace=True)
print("Dropped rows with missing values in key columns.")

# --- VERIFICATION ---
# Verify the shape of the DataFrame after cleaning
print(f"Shape after cleaning: {df.shape}")

# Verify that there are no more missing values
print("\nMissing values count after cleaning:")
print(df.isnull().sum().sum())

Shape before cleaning: (843, 69)
Dropped 'Comorbidities' column.
Dropped rows with missing values in key columns.
Shape after cleaning: (842, 68)

Missing values count after cleaning:
0


In [5]:
# Feature Engineering: Convert 'Age' column from ranges to a numerical value

# First, let's create a copy of our DataFrame to avoid any warnings
df_processed = df.copy()

# This function will convert an age string (e.g., '60 - 69') to its average (65)
def convert_age(age_str):
    if age_str == '90 or older':
        return 90
    else:
        # Split the string '60 - 69' into ['60', '-', '69'], take the first part '60' and add 5
        return int(age_str.split(' ')[0]) + 5

# Apply this function to the entire 'Age' column
df_processed['Age'] = df_processed['Age'].apply(convert_age)


# --- VERIFICATION ---
# Display the first few values of the updated 'Age' column to see the change
print("Data types of columns after converting Age:")
print(df_processed[['Age', 'Gender', 'Race']].dtypes)

print("\nFirst 5 rows of the updated 'Age' column:")
print(df_processed['Age'].head())

Data types of columns after converting Age:
Age        int64
Gender    object
Race      object
dtype: object

First 5 rows of the updated 'Age' column:
0    65
1    65
2    65
3    65
4    65
Name: Age, dtype: int64


In [6]:
import pandas as pd

# Let's find the exact names of the key genetic columns
cyp2c9_col = 'CYP2C9 genotypes'
# The VKORC1 column name is very long, so we find it programmatically
vkorc1_col = [col for col in df_processed.columns if 'VKORC1 genotype: -1639 G>A' in col][0]

# The two main genetic columns we need to convert
genetic_cols_to_encode = [cyp2c9_col, vkorc1_col]

print(f"Encoding the following columns:\n1. {cyp2c9_col}\n2. {vkorc1_col}\n")

# Apply one-hot encoding using pd.get_dummies
df_genetics_encoded = pd.get_dummies(df_processed[genetic_cols_to_encode])

# Join the new encoded columns back to our main dataframe
df_processed = pd.concat([df_processed, df_genetics_encoded], axis=1)

# Drop the original text-based genetic columns now that we have the encoded versions
df_processed.drop(columns=genetic_cols_to_encode, inplace=True)


# --- VERIFICATION ---
# Display the first 5 rows of our dataframe to see the new columns
print("DataFrame head with new genetic columns:")
display(df_processed.head())

print(f"\nTotal number of columns has now increased to: {df_processed.shape[1]}")

Encoding the following columns:
1. CYP2C9 genotypes
2. VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231

DataFrame head with new genetic columns:


,PharmGKB Subject ID,Gender,Race,Ethnicity,Age,Height (cm),Weight (kg),Indication for Warfarin Treatment,Medications,Target INR,...,VKORC1-1639.7,VKORC1-1639.8,VKORC1-1639.9,VKORC1-1639.10,CYP2C9 genotypes_CYP2C9 *1/*1,CYP2C9 genotypes_CYP2C9 *1/*2,CYP2C9 genotypes_CYP2C9 *1/*3,VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231_A/A,VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231_A/G,VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231_G/G
0,PA128919639,male,White,Not Hispanic or Latino,65,170.18,103.4,Pulmonary Embolism,levothyroxine;metformin,2.5,...,0,0,0.0,0.0,False,True,False,False,True,False
1,PA128919639,male,White,Not Hispanic or Latino,65,170.18,103.4,Pulmonary Embolism,levothyroxine;metformin,2.5,...,0,0,0.0,0.0,False,True,False,False,True,False
2,PA128919639,male,White,Not Hispanic or Latino,65,170.18,103.4,Pulmonary Embolism,levothyroxine;metformin,2.5,...,0,0,0.0,0.0,False,True,False,False,True,False
3,PA128919639,male,White,Not Hispanic or Latino,65,170.18,103.4,Pulmonary Embolism,levothyroxine;metformin,2.5,...,0,0,0.0,0.0,False,True,False,False,True,False
4,PA128919639,male,White,Not Hispanic or Latino,65,170.18,103.4,Pulmonary Embolism,levothyroxine;metformin,2.5,...,0,0,0.0,0.0,False,True,False,False,True,False



Total number of columns has now increased to: 72


In [7]:
# Identify remaining categorical columns to encode
cols_to_encode = [
    'Race',
    'Ethnicity',
    'Indication for Warfarin Treatment',
    'Gender'
]

print("Encoding the following columns with one-hot encoding:")
for col in cols_to_encode:
    print(f"- {col}")

# Apply one-hot encoding
df_encoded = pd.get_dummies(df_processed[cols_to_encode])

# Join the new encoded columns back to our main dataframe
df_processed = pd.concat([df_processed, df_encoded], axis=1)

# Drop the original text-based columns
df_processed.drop(columns=cols_to_encode, inplace=True)

# --- VERIFICATION ---
print(f"\nDataFrame now has {df_processed.shape[1]} columns.")
print("Verifying the first 5 rows with all new columns:")
display(df_processed.head())

Encoding the following columns with one-hot encoding:
- Race
- Ethnicity
- Indication for Warfarin Treatment
- Gender

DataFrame now has 76 columns.
Verifying the first 5 rows with all new columns:


,PharmGKB Subject ID,Age,Height (cm),Weight (kg),Medications,Target INR,Measured INR,Therapeutic Dose of Warfarin,INR on Reported Therapeutic Dose of Warfarin,Current Smoker,...,VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231_A/G,VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231_G/G,Race_White,Ethnicity_Not Hispanic or Latino,Indication for Warfarin Treatment_Atrial Fibrillation,Indication for Warfarin Treatment_Deep Vein Thrombosis,Indication for Warfarin Treatment_Mechanical Heart Valve,Indication for Warfarin Treatment_Pulmonary Embolism,Gender_female,Gender_male
0,PA128919639,65,170.18,103.4,levothyroxine;metformin,2.5,2.7,49.0,2.7,0,...,True,False,True,True,False,False,False,True,False,True
1,PA128919639,65,170.18,103.4,levothyroxine;metformin,2.5,2.7,49.0,2.7,0,...,True,False,True,True,False,False,False,True,False,True
2,PA128919639,65,170.18,103.4,levothyroxine;metformin,2.5,2.7,49.0,2.7,0,...,True,False,True,True,False,False,False,True,False,True
3,PA128919639,65,170.18,103.4,levothyroxine;metformin,2.5,2.7,49.0,2.7,0,...,True,False,True,True,False,False,False,True,False,True
4,PA128919639,65,170.18,103.4,levothyroxine;metformin,2.5,2.7,49.0,2.7,0,...,True,False,True,True,False,False,False,True,False,True


In [8]:
# Identify any remaining non-numerical columns (like IDs and complex text)
object_cols = df_processed.select_dtypes(include=['object']).columns
print(f"Remaining non-numerical columns to be dropped: \n{list(object_cols)}")

# Drop these final columns
df_processed.drop(columns=object_cols, inplace=True)

# --- FINAL PREPARATION: Define Features (X) and Target (y) ---

# Our target 'y' is the column we want to predict
y = df_processed['Therapeutic Dose of Warfarin']

# Our features 'X' are all the other columns
X = df_processed.drop(columns=['Therapeutic Dose of Warfarin'])

# --- VERIFICATION ---
print(f"\nFinal shape of feature data (X): {X.shape}")
print(f"Final shape of target data (y): {y.shape}")
print("\nFirst 5 rows of the final features (X):")
display(X.head())

print("\nFirst 5 values of the final target (y):")
print(y.head())

Remaining non-numerical columns to be dropped: 
['PharmGKB Subject ID', 'Medications', 'Genotyped QC', 'VKORC1 QC genotype: -1639 G>A (3673); chr16:31015190; rs9923231', 'VKORC1 genotype: 497T>G (5808); chr16:31013055; rs2884737', 'VKORC1 QC genotype: 497T>G (5808); chr16:31013055; rs2884737', 'VKORC1 genotype: 1173 C>T(6484); chr16:31012379; rs9934438', 'VKORC1 QC genotype: 1173 C>T(6484); chr16:31012379; rs9934438', 'VKORC1 genotype: 1542G>C (6853); chr16:31012010; rs8050894', 'VKORC1 QC genotype: 1542G>C (6853); chr16:31012010; rs8050894', 'VKORC1 genotype: 3730 G>A (9041); chr16:31009822; rs2359612', 'VKORC1 QC genotype: 3730 G>A (9041); chr16:31009822; rs2359612', 'VKORC1 genotype: 2255C>T (7566); chr16:31011297; rs7294', 'VKORC1 QC genotype: 2255C>T (7566); chr16:31011297; rs7294', 'VKORC1 genotype: -4451 C>A (861); chr16:31018002; rs17708472', 'VKORC1 QC genotype: -4451 C>A (861); chr16:31018002; rs17708472', 'CYP2C9 consensus', 'VKORC1 -1639 G>A consensus', 'VKORC1 497 T>G cons

,Age,Height (cm),Weight (kg),Target INR,Measured INR,INR on Reported Therapeutic Dose of Warfarin,Current Smoker,CYP2C9 *1/*1,CYP2C9 *1/*2,CYP2C9 *1/*3,...,VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231_A/G,VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231_G/G,Race_White,Ethnicity_Not Hispanic or Latino,Indication for Warfarin Treatment_Atrial Fibrillation,Indication for Warfarin Treatment_Deep Vein Thrombosis,Indication for Warfarin Treatment_Mechanical Heart Valve,Indication for Warfarin Treatment_Pulmonary Embolism,Gender_female,Gender_male
0,65,170.18,103.4,2.5,2.7,2.7,0,0,1,0,...,True,False,True,True,False,False,False,True,False,True
1,65,170.18,103.4,2.5,2.7,2.7,0,0,1,0,...,True,False,True,True,False,False,False,True,False,True
2,65,170.18,103.4,2.5,2.7,2.7,0,0,1,0,...,True,False,True,True,False,False,False,True,False,True
3,65,170.18,103.4,2.5,2.7,2.7,0,0,1,0,...,True,False,True,True,False,False,False,True,False,True
4,65,170.18,103.4,2.5,2.7,2.7,0,0,1,0,...,True,False,True,True,False,False,False,True,False,True



First 5 values of the final target (y):
0    49.0
1    49.0
2    49.0
3    49.0
4    49.0
Name: Therapeutic Dose of Warfarin, dtype: float64


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Step 1: Split the data into training (80%) and testing (20%) sets
# random_state=42 is a number we use to ensure the split is the same every time we run the code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Initialize and train the Linear Regression model
# The .fit() function is where the model 'learns' from the data
print("Training the Linear Regression model...")
model = LinearRegression()
model.fit(X_train, y_train)
print("Model training complete.")

# Step 3: Make predictions on the unseen test data
y_pred = model.predict(X_test)

# Step 4: Evaluate the model's performance
# We use Root Mean Squared Error (RMSE) to see how far off our predictions are on average.
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"\n--- Model Evaluation ---")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Training the Linear Regression model...
Model training complete.

--- Model Evaluation ---
Root Mean Squared Error (RMSE): 4.64


In [10]:
import joblib

# Define the path and filename for our model
model_filename = '../models/linear_regression_base_v1.pkl'

# Save the trained model to the file
joblib.dump(model, model_filename)

print(f"Model saved successfully to: {model_filename}")

Model saved successfully to: ../models/linear_regression_base_v1.pkl


In [11]:
# Load the model from the file
loaded_model = joblib.load('../models/linear_regression_base_v1.pkl')

# Let's test it by making a prediction on the first row of our test data
first_patient_data = X_test.iloc[0]
predicted_dose = loaded_model.predict([first_patient_data])

print("Successfully loaded the model and made a test prediction.")
print(f"Data for first patient in test set: \n{first_patient_data}")
print(f"\nPredicted dose for this patient: {predicted_dose[0]:.2f} mg/week")

Successfully loaded the model and made a test prediction.
Data for first patient in test set: 
Age                                                                     65
Height (cm)                                                         167.64
Weight (kg)                                                          130.2
Target INR                                                             2.5
Measured INR                                                           2.7
INR on Reported Therapeutic Dose of Warfarin                           2.7
Current Smoker                                                           0
CYP2C9 *1/*1                                                             1
CYP2C9 *1/*2                                                             0
CYP2C9 *1/*3                                                             0
CYP2C9 *2/*2                                                             0
CYP2C9 *2/*3                                                             0
CYP2C

c:\Users\aniru\OneDrive\Desktop\DosagePredictAI\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [12]:
# Clean up the column names in X
X.columns = ["".join (c if c.isalnum() else '_' for c in str(x)) for x in X.columns]

# Let's check the first few new column names to see the change
print("Cleaned feature names (first 10):")
print(X.columns[:10].tolist())

Cleaned feature names (first 10):
['Age', 'Height__cm_', 'Weight__kg_', 'Target_INR', 'Measured_INR', 'INR_on_Reported_Therapeutic_Dose_of_Warfarin', 'Current_Smoker', 'CYP2C9__1__1', 'CYP2C9__1__2', 'CYP2C9__1__3']


In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Step 1: Split the data again (using our cleaned 'X')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Initialize and train the Random Forest model
# n_estimators=100 means we are creating a 'committee' of 100 decision trees
print("Training the Random Forest model...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
print("Model training complete.")

# Step 3: Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Step 4: Evaluate the new model's performance
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"\n--- Model Evaluation ---")
print(f"Linear Regression RMSE (Baseline): 4.64")
print(f"Random Forest Regressor RMSE:      {rmse_rf:.2f}")

Training the Random Forest model...
Model training complete.

--- Model Evaluation ---
Linear Regression RMSE (Baseline): 4.64
Random Forest Regressor RMSE:      0.14


In [14]:
import joblib

# Define the path and filename for our new model
model_filename = '../models/random_forest_enhanced_v1.pkl'

# Save the trained Random Forest model to the file
# Note: We are saving the 'rf_model' variable this time
joblib.dump(rf_model, model_filename)

print(f"Random Forest model saved successfully to: {model_filename}")

Random Forest model saved successfully to: ../models/random_forest_enhanced_v1.pkl


In [15]:
# --- Building a TRUE 'Clinical-Only' Base Model ---

# A better way to identify all genomic columns (anything with CYP2C9 or VKORC1 in the name)
genomic_cols_to_drop = [col for col in X.columns if 'CYP2C9' in col or 'VKORC1' in col]

X_clinical_true = X.drop(columns=genomic_cols_to_drop)

print(f"Number of original features: {X.shape[1]}")
print(f"Number of genomic features to drop: {len(genomic_cols_to_drop)}")
print(f"Number of remaining true clinical features: {X_clinical_true.shape[1]}")

# Split the TRUE clinical-only data
X_train_cli, X_test_cli, y_train_cli, y_test_cli = train_test_split(X_clinical_true, y, test_size=0.2, random_state=42)

# Train a new Random Forest model on this corrected data
print("\nTraining the TRUE Clinical-Only Random Forest model...")
rf_model_clinical_true = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_clinical_true.fit(X_train_cli, y_train_cli)
print("Model training complete.")

# --- RE-SAVE THE BASE MODEL AND ITS COLUMNS ---
# This will overwrite our previous, slightly incorrect base model
joblib.dump(rf_model_clinical_true, '../models/random_forest_base_v1.pkl')
joblib.dump(X_clinical_true.columns.tolist(), '../models/base_model_columns.pkl')
print("\nCorrected Base Model and its columns have been saved.")

Number of original features: 49
Number of genomic features to drop: 34
Number of remaining true clinical features: 15

Training the TRUE Clinical-Only Random Forest model...
Model training complete.

Corrected Base Model and its columns have been saved.


In [16]:
import joblib

# Define the path and filename for our base model
model_filename = '../models/random_forest_base_v1.pkl'

# Save the trained clinical-only model to the file
joblib.dump(rf_model_clinical, model_filename)

print(f"Clinical-Only Base Model saved successfully to: {model_filename}")

NameError: name 'rf_model_clinical' is not defined

In [ ]:
import joblib

# Save the first row of your test set as a sample for the API
# We use X_test because the model has never been trained on this data.
joblib.dump(X_test.iloc[0:1], '../models/sample_patient_test.pkl')

print("Sample patient data saved to 'models/sample_patient_test.pkl'")

Sample patient data saved to 'models/sample_patient_test.pkl'


In [ ]:
import joblib

# Define the path to save the column list (using the correct relative path)
column_list_path = '../models/model_columns.pkl'

# Save the list of columns from your final feature DataFrame 'X'
joblib.dump(X.columns.tolist(), column_list_path)

print(f"Model columns saved successfully to: {column_list_path}")

Model columns saved successfully to: ../models/model_columns.pkl


In [ ]:
import joblib

# X_clinical is the DataFrame with only clinical features we created earlier
# If this variable is not in memory, you may need to "Run All Above"

# Define the path and save the column list
base_columns_path = '../models/base_model_columns.pkl'
joblib.dump(X_clinical.columns.tolist(), base_columns_path)

print(f"Base model columns saved successfully to: {base_columns_path}")

Base model columns saved successfully to: ../models/base_model_columns.pkl


In [ ]:
!pip install pandas numpy scikit-learn matplotlib seaborn


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
